In [1]:
import os
import time
from pyspark.sql.utils import AnalysisException
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SQLContext, DataFrame, SparkSession
from pyspark.sql.functions import size, lit, explode, col, round, dense_rank, rank, desc
from pyspark.sql.window import Window
import json
from functools import reduce

In [2]:
pyspark_session_sql = SparkSession \
                    .builder \
                    .appName("spark_session_sql") \
                    .config('spark.driver.memory', '6g') \
                    .getOrCreate()


sqlcontext = SQLContext(pyspark_session_sql)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
def get_csv_file(file, sep=None):
    print('\n\x1b[1;33;40mRead Csv Data...\x1b[0m\n')
    csv_file  = pyspark_session_sql.read.csv(file, header = True, sep = sep)
    return csv_file

def get_dataframe_table(context, data, table_name):
    context.registerDataFrameAsTable(data, table_name)

In [4]:
data_per = get_csv_file('csvfiles/Person_StateProvince.csv', sep = ';')
data_per.show(5)


Read Csv Data...

+---------------+-----------------+-----------------+-----------------------+--------------+-----------+--------------------+--------------------+
|StateProvinceID|StateProvinceCode|CountryRegionCode|IsOnlyStateProvinceFlag|          Name|TerritoryID|             rowguid|        ModifiedDate|
+---------------+-----------------+-----------------+-----------------------+--------------+-----------+--------------------+--------------------+
|              1|              AB |               CA|                      0|       Alberta|          6|298C2880-AB1C-498...|2014-02-08 10:17:...|
|              2|              AK |               US|                      0|        Alaska|          1|5B7B8462-A888-4E0...|2014-02-08 10:17:...|
|              3|              AL |               US|                      0|       Alabama|          5|41B328BE-21AE-45D...|2014-02-08 10:17:...|
|              4|              AR |               US|                      0|      Arkansas|       

In [5]:
data_sal = get_csv_file('csvfiles/Sales_SalesTaxRate.csv', sep = ';')
data_sal.show(5)


Read Csv Data...

+--------------+---------------+-------+-------+--------------------+--------------------+--------------------+
|SalesTaxRateID|StateProvinceID|TaxType|TaxRate|                Name|             rowguid|        ModifiedDate|
+--------------+---------------+-------+-------+--------------------+--------------------+--------------------+
|             1|              1|      1|  14,00|Canadian GST + Al...|683DE5DD-521A-47D...|2008-04-30 00:00:...|
|             2|             57|      1|  14,25|Canadian GST + On...|05C4FFDB-4F84-4CD...|2008-04-30 00:00:...|
|             3|             63|      1|  14,25|Canadian GST + Qu...|D4EDB557-56D7-403...|2008-04-30 00:00:...|
|             4|              1|      2|   7,00|        Canadian GST|F0D76907-B433-453...|2008-04-30 00:00:...|
|             5|             57|      2|   7,00|        Canadian GST|7E0E97A2-878B-476...|2008-04-30 00:00:...|
+--------------+---------------+-------+-------+--------------------+----------------

In [6]:
get_dataframe_table(sqlcontext, data_per, "data_per")
get_dataframe_table(sqlcontext, data_sal, "data_sal")

In [7]:
table_avg_tax = sqlcontext.sql('''WITH table_avg_tax AS (
                                   SELECT data_sal.StateProvinceID, data_sal.TaxRate, data_per.StateProvinceID, data_per.CountryRegionCode
                                   FROM data_sal 
                                   LEFT JOIN data_per 
                                   ON data_sal.StateProvinceID = data_per.StateProvinceID)
                                SELECT table_avg_tax.CountryRegionCode, ROUND(AVG(REPLACE(table_avg_tax.TaxRate,',','.')),4) AS average_taxRate 
                                FROM table_avg_tax 
                                GROUP BY table_avg_tax.CountryRegionCode
                                ORDER BY table_avg_tax.CountryRegionCode''')
table_avg_tax.show()

+-----------------+---------------+
|CountryRegionCode|average_taxRate|
+-----------------+---------------+
|               AU|           10.0|
|               CA|         8.4333|
|               DE|           16.0|
|               FR|           19.6|
|               GB|           17.5|
|               US|          7.405|
+-----------------+---------------+



In [8]:
data_country = get_csv_file('csvfiles/Sales_CountryRegionCurrency.csv', sep = ';')
data_country.show(5)


Read Csv Data...

+-----------------+------------+--------------------+
|CountryRegionCode|CurrencyCode|        ModifiedDate|
+-----------------+------------+--------------------+
|               AE|         AED|2014-02-08 10:17:...|
|               AR|         ARS|2014-02-08 10:17:...|
|               AT|         ATS|2014-02-08 10:17:...|
|               AT|         EUR|2008-04-30 00:00:...|
|               AU|         AUD|2014-02-08 10:17:...|
+-----------------+------------+--------------------+
only showing top 5 rows



In [9]:
data_country_reg = get_csv_file('csvfiles/Person_CountryRegion.csv', sep = ';')
data_country_reg.show(5)


Read Csv Data...

+-----------------+--------------------+--------------------+
|CountryRegionCode|                Name|        ModifiedDate|
+-----------------+--------------------+--------------------+
|               AD|             Andorra|2008-04-30 00:00:...|
|               AE|United Arab Emirates|2008-04-30 00:00:...|
|               AF|         Afghanistan|2008-04-30 00:00:...|
|               AG| Antigua and Barbuda|2008-04-30 00:00:...|
|               AI|            Anguilla|2008-04-30 00:00:...|
+-----------------+--------------------+--------------------+
only showing top 5 rows



In [10]:
data_currency_rate = get_csv_file('csvfiles/Sales_CurrencyRate.csv', sep = ';')
data_currency_rate.show(5)


Read Csv Data...

+--------------+--------------------+----------------+--------------+-----------+------------+--------------------+
|CurrencyRateID|    CurrencyRateDate|FromCurrencyCode|ToCurrencyCode|AverageRate|EndOfDayRate|        ModifiedDate|
+--------------+--------------------+----------------+--------------+-----------+------------+--------------------+
|             1|2011-05-31 00:00:...|             USD|           ARS|       1,00|      1,0002|2011-05-31 00:00:...|
|             2|2011-05-31 00:00:...|             USD|           AUD|     1,5491|        1,55|2011-05-31 00:00:...|
|             3|2011-05-31 00:00:...|             USD|           BRL|     1,9379|      1,9419|2011-05-31 00:00:...|
|             4|2011-05-31 00:00:...|             USD|           CAD|     1,4641|      1,4683|2011-05-31 00:00:...|
|             5|2011-05-31 00:00:...|             USD|           CNY|     8,2781|      8,2784|2011-05-31 00:00:...|
+--------------+--------------------+----------------

In [11]:
data_currency = get_csv_file('csvfiles/Sales_Currency.csv', sep = ';')
data_currency.show(5)


Read Csv Data...

+------------+--------------------+--------------------+
|CurrencyCode|                Name|        ModifiedDate|
+------------+--------------------+--------------------+
|         AED|      Emirati Dirham|2008-04-30 00:00:...|
|         AFA|             Afghani|2008-04-30 00:00:...|
|         ALL|                 Lek|2008-04-30 00:00:...|
|         AMD|       Armenian Dram|2008-04-30 00:00:...|
|         ANG|Netherlands Antil...|2008-04-30 00:00:...|
+------------+--------------------+--------------------+
only showing top 5 rows



In [12]:
get_dataframe_table(sqlcontext, data_country_reg, "data_country_reg")
get_dataframe_table(sqlcontext, data_country, "data_country")
get_dataframe_table(sqlcontext, data_currency, "data_currency")
get_dataframe_table(sqlcontext, data_currency_rate, "data_currency_rate")

querypy = sqlcontext.sql('''WITH tablefinal AS (
                                SELECT data_country_reg.Name, 
                                      data_country_reg.CountryRegionCode, 
                                      data_country.CurrencyCode,
                                      data_currency.Name AS currency_name,
                                      data_currency_rate.AverageRate,
                                      data_per.StateProvinceID,
                                      data_sal.TaxRate 
                               FROM data_country_reg 
                               INNER JOIN data_country 
                               ON data_country_reg.CountryRegionCode = data_country.CountryRegionCode
                               INNER JOIN data_currency
                               ON data_country.CurrencyCode = data_currency.CurrencyCode
                               INNER JOIN data_currency_rate
                               ON data_country.CurrencyCode = data_currency_rate.ToCurrencyCode
                               INNER JOIN data_per
                               ON data_country.CountryRegionCode = data_per.CountryRegionCode
                               INNER JOIN data_sal
                               ON data_per.StateProvinceID = data_sal.StateProvinceID)
                            SELECT tablefinal.Name AS country_name,
                                   tablefinal.currency_name AS currency_name,
                                   MAX(ROUND(REPLACE(tablefinal.AverageRate,',','.'),2)) AS currency_rate,
                                   ROUND(AVG(REPLACE(tablefinal.TaxRate,',','.')),2) AS average_tax_rate
                            FROM tablefinal
                            GROUP BY tablefinal.Name, tablefinal.currency_name
                            ORDER BY tablefinal.Name''')
querypy.show(15)

+--------------+--------------------+-------------+----------------+
|  country_name|       currency_name|currency_rate|average_tax_rate|
+--------------+--------------------+-------------+----------------+
|     Australia|   Australian Dollar|         2.09|            10.0|
|        Canada|     Canadian Dollar|         1.62|            8.43|
|        France|        French Franc|         7.37|            19.6|
|        France|                EURO|         1.21|            19.6|
|       Germany|       Deutsche Mark|          2.2|            16.0|
|       Germany|                EURO|         1.21|            16.0|
|United Kingdom|United Kingdom Pound|         0.73|            17.5|
| United States|           US Dollar|          1.0|            7.41|
+--------------+--------------------+-------------+----------------+



In [13]:
pyspark_session_sql.stop()